In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="KwGGfBeUSV6vpvnm5YeM")
project = rf.workspace("my-projects-bschq").project("ppe_v3-vkug4")
version = project.version(1)
dataset = version.download("yolov11")              

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 38.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 40.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 99.0 MB/s eta 0:00:00:00:01
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigqu


Extracting Dataset Version Zip to ppe_v3-1 in yolov11:: 100%|██████████| 4560/4560 [00:00<00:00, 8384.95it/s]


In [2]:
!pip install ultralytics 

import os
os.environ['WANDB_DISABLED'] = 'true'
os.environ['WANDB_MODE'] = 'disabled'

print("Libraries Installed Successfully")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.8 MB/s eta 0:00:00a 0:00:01
Libraries Installed Successfully


In [3]:
# ========== to calculate each class number ==========
import os
from collections import Counter

splits = ["train", "valid", "test"]
dataset_root = "/kaggle/working/ppe_v3-1"

total_counter = Counter()

for split in splits:
    labels_path = os.path.join(dataset_root, split, "labels")
    if not os.path.exists(labels_path):
        continue

    split_counter = Counter()

    for file in os.listdir(labels_path):
        if file.endswith(".txt"):
            with open(os.path.join(labels_path, file), "r") as f:
                for line in f:
                    class_id = int(line.split()[0])
                    split_counter[class_id] += 1
                    total_counter[class_id] += 1

    print(f"\n{split.upper()}")
    for k, v in sorted(split_counter.items()):
        print(f"Class {k}: {v}")

print("\nTOTAL")
for k, v in sorted(total_counter.items()):
    print(f"Class {k}: {v}")



TRAIN
Class 0: 1950
Class 1: 693
Class 2: 3
Class 3: 1787
Class 4: 1419
Class 5: 1030
Class 6: 2695
Class 7: 3855
Class 8: 1336

VALID
Class 0: 532
Class 1: 177
Class 2: 2
Class 3: 535
Class 4: 361
Class 5: 292
Class 6: 725
Class 7: 1026
Class 8: 389

TEST
Class 0: 114
Class 1: 44
Class 3: 101
Class 4: 70
Class 5: 84
Class 6: 147
Class 7: 217
Class 8: 73

TOTAL
Class 0: 2596
Class 1: 914
Class 2: 5
Class 3: 2423
Class 4: 1850
Class 5: 1406
Class 6: 3567
Class 7: 5098
Class 8: 1798


In [4]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt")
results = model.train(data ="/kaggle/working/ppe_v3-1/data.yaml" , epochs = 80 , imgsz = 640 , batch = 16)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.248 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/ppe_v3-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, k

In [5]:
from ultralytics import YOLO

model = YOLO("/kaggle/working/runs/detect/train/weights/best.pt")

metrics = model.val(
    data="/kaggle/working/ppe_v3-1/data.yaml",
    split="test"
)
print(metrics)

Ultralytics 8.3.248 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,583,907 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1000.8±461.8 MB/s, size: 56.1 KB)
val: Scanning /kaggle/working/ppe_v3-1/test/labels... 100 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 100/100 1.4Kit/s 0.1s
val: New cache created: /kaggle/working/ppe_v3-1/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 2.4it/s 2.9s0.2s
                   all        100        850      0.794       0.77      0.814       0.45
           face_nomask         57        114      0.804      0.866      0.903      0.456
            face_wmask         23         44      0.884      0.864      0.954      0.595
          hand_noglove         40        101      0.673      0.604      0.587      0.249
           hand_wglove         30         70      0.659      0.386